In [43]:
import pandas as pd

data_dir = "./Other_Datasets/ANERcorp/"
name = "train"

bad_words = {'ARTICLE', 'REVISIT', 'ARTICLE', 'article', 'REVIIST', 'ARTICE', 'ARTICLE', 'FIX', 'ID', ' ', 'REVISSIT', 'REVISIIT'}

sentences = []
sentences_labels = []

with open(data_dir + f"{name}.txt", 'r') as file:
    sentence = []  # Initialize as a list
    each_word_label = []
    for i, line in enumerate(file):
        # Check if the line is not empty
        if line.strip():
            parts = line.split(' ')
            # Check if parts has enough elements
            if len(parts) > 1:
                word, label = parts[0], parts[1].strip()
                # Check if the word is not in the set of bad words
                if label not in bad_words:
                    sentence.append(word)
                    each_word_label.append(label)

                # If the word is a period, it indicates the end of a sentence
                if '.' == word.strip():
                    if sentence:
                        sentences.append(' '.join(sentence))
                        sentences_labels.append(each_word_label)
                        sentence = []
                        each_word_label = []

# Now you have sentences and their corresponding labels
df = pd.DataFrame({'sentence': sentences, 'labels': sentences_labels})


In [44]:
df.loc[1]['sentence'] + '؟'

'وقال رئيس الاتحاد برند جوتشولك عند إعلان آخر تقرير سنوي للاتحاد إن مستقبل السوق مازال يفتقر إلي الخطوط الواضحة .؟'

In [45]:
new_df = pd.DataFrame()

for i in range(len(df)):
    words = df['sentence'][i].split(' ')
    labels = df['labels'][i]
    num_words = len(words)
    
    for j in range(num_words):
        new_row = pd.DataFrame({'label': [labels[j]], 'word': [words[j]], 'sentence': df['sentence'][i]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)


In [46]:
new_df['label'].unique()

array(['B-LOC', 'B-ORG', 'I-ORG', 'O', 'B-PERS', 'I-PERS', 'I-LOC',
       'B-MISC', 'I-MISC'], dtype=object)

In [47]:
new_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)

In [48]:
x = 67

y =  ' [MASK]'+ new_df.loc[x]['word'] + ' النوع؟' 
y

' [MASK]أن النوع؟'

In [49]:
x = 67

y =   new_df.loc[x]['word'] + ' type:'+ '[MASK]' 
y

'أن type:[MASK]'

In [50]:
type_df = new_df.copy()
label_mapping = {
    'B-LOC': 'LOC',
    'I-LOC': 'LOC',
    'B-PERS': 'PERS',
    'I-PERS': 'PERS',
    'B-ORG': 'ORG',
    'I-ORG': 'ORG',
    'B-MISC': 'MISC',
    'I-MISC': 'MISC',
    'O': 'O'
}

# Apply the mapping
type_df['label'] = new_df['label'].map(label_mapping)

In [51]:
location_df = new_df.copy()
label_mapping = {
    'B-LOC': 'Begin',
    'I-LOC': 'Inside',
    'B-PERS': 'Begin',
    'I-PERS': 'Inside',
    'B-ORG': 'Begin',
    'I-ORG': 'Inside',
    'B-MISC': 'Begin',
    'I-MISC': 'Inside',
    'O': 'O'
}

# Apply the mapping
location_df['label'] = new_df['label'].map(label_mapping)

In [52]:
type_df['label'].unique(),location_df['label'].unique()

(array(['LOC', 'ORG', 'O', 'PERS', 'MISC'], dtype=object),
 array(['Begin', 'Inside', 'O'], dtype=object))

In [53]:
data_dir = "./Other_Datasets/ANERcorpType/"

type_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)

In [54]:
data_dir = "./Other_Datasets/ANERcorpLocation/"

location_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)


# Truncating data

In [14]:
import pandas as pd
data_dir = "./Other_Datasets/ANERcorpType/"
name = "test"

df = pd.read_csv(data_dir + f"{name}.csv", header=None,names=['label','words','sentence'])

In [15]:
num_samples = 5000/5
num_samples

1000.0

In [16]:
df['label'].value_counts()

label
O       21313
PERS     1525
ORG       881
LOC       804
MISC      486
Name: count, dtype: int64

In [17]:
def sample_rows(group):
    return group.sample(min(500, len(group)))

# Use groupby and apply to select 500 rows from each label
sampled_data = df.groupby('label', group_keys=False).apply(sample_rows)


In [21]:
sampled_data = sampled_data.sample(frac = 1)


In [23]:
sampled_data.to_csv(data_dir + "test.csv", index=False, header=False)

Removing MISC for zero shot

In [21]:
import pandas as pd
data_dir = "./Other_Datasets/ANERcorpType/"
dataset = "dev"
df = pd.read_csv(data_dir + dataset + '.csv', header=None,names=['label','words','sentence'])

In [22]:
df.shape

(15014, 3)

In [23]:
df = df[df['label'] != 'MISC']

In [24]:
df.shape


(14842, 3)

In [25]:
df['label'].value_counts()

label
O       12942
PERS      741
ORG       727
LOC       432
Name: count, dtype: int64

In [26]:
output_dir = "./Other_Datasets/ANERcorpZero/"
df.to_csv( output_dir + dataset + '.csv', index=False, header=False)